# **Формируем базу данных**

**Задается количество торговых роботов**

In [11]:
# Сколько надо Юзеров
# Определяем количество элементов в USERS
try:
    a = int(input("Сколько участников торгов? \n"))
    USERS = ["User_{}".format(i) for i in range(1, a + 1)]
except:
    print("Введите число")

Сколько участников торгов? 
4


**Формируем файл исторических цен за 5 дней**

In [2]:
import os
import pandas as pd
import numpy as np

df1 = pd.read_csv("OrderLog20151207.txt")
df1 = df1[(df1['SECCODE'] == "SBER") & (df1['ACTION'] == 1) & (df1['BUYSELL'] == "B")]
df1['DAY'] = 1
df2 = pd.read_csv("OrderLog20151208.txt")
df2 = df2[(df2['SECCODE'] == "SBER") & (df1['ACTION'] == 1) & (df1['BUYSELL'] == "B")]
df2['DAY'] = 2
df3 = pd.read_csv("OrderLog20151209.txt")
df3 = df3[(df3['SECCODE'] == "SBER") & (df1['ACTION'] == 1) & (df1['BUYSELL'] == "B")]
df3['DAY'] = 3
df4 = pd.read_csv("OrderLog20151210.txt")
df4 = df4[(df4['SECCODE'] == "SBER") & (df1['ACTION'] == 1) & (df1['BUYSELL'] == "B")]
df4['DAY'] = 4
df5 = pd.read_csv("OrderLog20151211.txt")
df5 = df5[(df5['SECCODE'] == "SBER") & (df1['ACTION'] == 1) & (df1['BUYSELL'] == "B")]
df5['DAY'] = 5

df_w = pd.concat([df1, df2, df3, df4, df5], keys = [1, 2, 3, 4, 5])

df_w['CURRENT_MINUTE'] = (df_w['TIME']) % (100 * 1000)
df_w = df_w.pivot_table(index = ['SECCODE', 'CURRENT_MINUTE'], values = ['PRICE'],
                      aggfunc={'PRICE': ['first', 'min', 'max', 'last']})
df_w = pd.DataFrame({'OPEN': df_w['PRICE']['first'],'LOW': df_w['PRICE']['min'],
                     'HIGH': df_w['PRICE']['max'], 'CLOSE': df_w['PRICE']['last'] })
df_w['TIME'] = range(1, len(df_w.index) + 1)

hist_price = df_w[['TIME', 'OPEN', 'LOW', 'HIGH', 'CLOSE']]
hist_price = hist_price.set_index('TIME')

hist_price.to_csv("hist_price.csv")

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


**Создаем все необходимые таблицы**

In [25]:
# Импортируем библиотеку pandas
import pandas as pd

# Импортируем модуль os
import os

hist_price = pd.read_csv('hist_price.csv')
Big_OrderBook = pd.read_csv('OrderLog20151207.csv')

Big_OrderBook = Big_OrderBook[Big_OrderBook['SECCODE'] == 'SBER']

# Функция преобразования времени из вида int как 100000000 в (10 * 60^2 + 0 * 60 + 0) * 10^6 + 0,
# во время 00:00:00.000 от начала торгов
def time_convertion(time):
    time_sting = str(time)
    micro_seconds = int(time_sting[-3:])
    seconds = int(time_sting[-5:-3])
    minutes = int(time_sting[-7:-5])
    hours = int(time_sting[:-7])
    
    result = (hours * 60**2 + minutes * 60 + seconds) * 10**6 + micro_seconds
    return result

# Применяем функцию преобразования времени 
Big_OrderBook['TIME_FROM_START'] = Big_OrderBook['TIME']
Big_OrderBook['TIME_FROM_START'] = pd.Series(map(time_convertion, Big_OrderBook['TIME_FROM_START']), index = Big_OrderBook.index)
Big_OrderBook['TIME_FROM_START'] = Big_OrderBook['TIME_FROM_START'] - Big_OrderBook['TIME_FROM_START'][Big_OrderBook.index[0]]


'''def time_convertion_sec(time):
    time_sting = str(time)
    micro_seconds = int(time_sting[-3:])
    seconds = int(time_sting[-5:-3])
    minutes = int(time_sting[-7:-5])
    hours = int(time_sting[:-7])
    
    result = (hours * 60**2 + minutes * 60 + seconds)
    return result

def time_convertion_min(time):
    time_sting = str(time)
    micro_seconds = int(time_sting[-3:])
    seconds = int(time_sting[-5:-3])
    minutes = int(time_sting[-7:-5])
    hours = int(time_sting[:-7])
    
    result = (hours * 60 + minutes)
    return result
Big_OrderBook = pd.read_csv('OrderLog20151207.txt')
Big_OrderBook = Big_OrderBook[(Big_OrderBook['SECCODE'] == 'SBER') &(Big_OrderBook['ACTION'] != 2)]
Big_OrderBook['TRADE_SEC'] = Big_OrderBook['TIME']
Big_OrderBook['TRADE_SEC'] = pd.Series(map(time_convertion_sec, Big_OrderBook['TRADE_SEC']), index = Big_OrderBook.index)
Big_OrderBook['TRADE_SEC'] = Big_OrderBook['TRADE_SEC'] - Big_OrderBook['TRADE_SEC'][Big_OrderBook.index[0]]

Big_OrderBook['TIME'] = pd.Series(map(time_convertion_min, Big_OrderBook['TIME']), index = Big_OrderBook.index)
Big_OrderBook['TIME'] = Big_OrderBook['TIME'] - Big_OrderBook['TIME'][Big_OrderBook.index[0]]'''


# Формируем базу данных для пользовательских роботов, включающую список текущих действий роботов и временных данных
start_account = {'cash' : 100000, 'assets' : 0,
                'limit_orders' : pd.DataFrame({'orderno': [], 'buysell': [],
                                               'price': [], 'volume': []})}
basic_actions = pd.DataFrame({'action': [], 'orderno': [],
                              'buysell': [], 'price': [], 'volume': []})

Accounts_Info = {USERS[i]: start_account for i in range(len(USERS))}
List_of_Actions = {USERS[i]: basic_actions for i in range(len(USERS))}
temp_data = {USERS[i]: {} for i in range(len(USERS))}

# Создаем книгу заявок
OrderBook = pd.DataFrame({'no': [0], 'time': [0], 'trade_sec': [0], 'ID': ["bot"],
                          'action': ['drop'], 'buysell': [0], 'orderno': [0],
                          'bot_orderno': [0], 'price': [0], 'volume': [0]
                          #'stringsAsFactors': ['F']
                          })
# Создаем книгу сделок
TradeBook = pd.DataFrame({'tradeno': [0], 'time': [0], 'trade_sec': [0], 'buy_ID': ["bot"],
                          'sell_ID': ['bot'], 'buy_orderno': [0], 'sell_orderno': [0],
                          'price': [0], 'volume': [0]
                          #'stringsAsFactors': ['F']
                          })
# Создаем датафрейм представлящий собой стакан заявок
DOM = {'supply': pd.DataFrame({'orderno': [0], 'ID': ["bot"],
                                'price': [0], 'volume': [0], 'bot_orderno': [0]
                                #'stringsAsFactors': ['F']
                               }),
       'demand': pd.DataFrame({'orderno': [0], 'ID': ["bot"],
                                'price': [0], 'volume': [0], 'bot_orderno': [0]
                                #'stringsAsFactors': ['F']
                               })
       }
# На основе стакана заявок формируем таблицу лучших цен спроса и предложения
bid_ask = pd.DataFrame({'time': [0],
                        'bid': [DOM['demand']['price'].max()],
                        'ask': [DOM['supply']['price'].min()]
                        })
# Заполняем таблицы, в которые позже будут добавляться данные по торгам
def rm_dom(N = 1e5, N2 = 1e4):
    DOM = {'supply': pd.DataFrame({'orderno': [0], 'bot_orderno': [0], 'ID': ["bot"],
                                   'price': [0], 'volume': [0]
                                   # 'stringsAsFactors': ['F']
                                   }),
           'demand': pd.DataFrame({'orderno': [0], 'bot_orderno': [0], 'ID': ["bot"],
                                   'price': [0], 'volume': [0]
                                   # 'stringsAsFactors': ['F']
                                   })
           }
    demand = DOM['demand']
    DOM['demand'] = demand[demand['volume'] != 0]
    supply = DOM['supply']
    DOM['supply'] = supply[supply['volume'] != 0]
    empty_array = [0 for i in range(int(N-1))]
    OrderBook = pd.DataFrame({
        'no': empty_array,
        'time': empty_array,
        'trade_sec': empty_array,
        'ID': empty_array,
        'action': ['NA' for i in range(int(N-1))],
        'buysell': ['X' for i in range(int(N-1))],
        'orderno': empty_array,
        'bot_orderno': empty_array,
        'price': empty_array,
        'volume': empty_array
    })

    TradeBook = pd.DataFrame({
        'tradeno': empty_array,
        'time': empty_array,
        'trade_sec': empty_array,
        'buy_ID': ['X' for i in range(int(N-1))],
        'sell_ID': ['X' for i in range(int(N-1))],
        'buy_orderno': empty_array,
        'sell_orderno': empty_array,
        'price': empty_array,
        'volume': empty_array
    })
    
    return OrderBook, TradeBook, DOM




# **Заполняем поля кодами стратегий-юзеров**

**Бот моментум по рыночным заявкам**

In [26]:
import pandas as pd
# import numpy as np
# import math
# import matplotlib.pyplot as plt
# import sklearn.linear_model as skl
# import datetime as dt
# import os

def strategy_M(hist_price, bid_ask, account_info=0,temp_data=0):
    a = hist_price # для компактности заменям hist_price на a
    a['return'] = a['CLOSE'] / a['CLOSE'].shift() # добавили колонку с доходностью (следующее значение делим на предыдущее)
    n = len(a.CLOSE) # количество строк в таблице 
    mean = a['return'][n - 20:n].mean() # скользящее среднее в колонке return 

    if account_info['assets'] == 0:
        if mean == 0: # если среднее равно нулю, то ничего не делаем в силу неопределённости
            action = pd.DataFrame({'action': [0],
                                   'orderno': [0],
                                   'buysell': [0],
                                   'price': [0],
                                   'volume': [0]})
            orders = {'ACTION': action, 'temp_data': 0}
        else:
            if mean > 0: # если среднее больше нуля, то покупаем в количестве, равном 100
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['B'],
                                       'price': [0],
                                       'volume': [100]})
                TP = a.loc[n - 1, 'CLOSE'] * 1.01 # если цена выросла более чем на 1%, то фиксируем прибыль
                SL = a.loc[n - 1, 'CLOSE'] * 0.99 # если цена упала более чем на 1%, то фиксируем убыток
                temp_data = {'TP': TP, 'SL': SL}
                orders = {'ACTION': action, 'temp_data': temp_data}
            if mean < 0:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['S'],
                                       'price': [0],
                                       'volume': [100]})
                TP = a.loc[n - 1, 'CLOSE'] * 1.01 # если цена выросла более чем на 1%, то фиксируем прибыль
                SL = a.loc[n - 1, 'CLOSE'] * 0.99 # если цена упала более чем на 1%, то фиксируем убыток
                temp_data = {'TP': TP, 'SL': SL} # таблица с take profit и stop loss
                orders = {'ACTION': action, 'temp_data': temp_data}

    else:
        if account_info['assets'] > 0:
            if a.loc[n - 1, 'CLOSE'] > temp_data['TP'] or a.loc[n - 1, 'CLOSE'] < temp_data['SL']:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['S'],
                                       'price': [0],
                                       'volume': [100]})
                orders = {'ACTION': action, 'temp_data': temp_data}
            else:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': [0],
                                       'price': [0],
                                       'volume': [0]})
                orders = {'ACTION': action, 'temp_data': temp_data}
        if account_info['assets'] < 0:
            if a.loc[n - 1, 'CLOSE'] > temp_data['TP'] or a.loc[n - 1, 'CLOSE'] < temp_data['SL']:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['B'],
                                       'price': [0],
                                       'volume': [100]})
                orders = {'ACTION': action, 'temp_data': temp_data}
            else:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': [0],
                                       'price': [0],
                                       'volume': [0]})
                orders = {'ACTION': action, 'temp_data': temp_data}
    return orders

**Робот 2**

In [27]:
def strategy_M2(hist_price, bid_ask, account_info=0,temp_data=0):
    a = hist_price # для компактности заменям hist_price на a
    a['return'] = a['CLOSE'] / a['CLOSE'].shift() # добавили колонку с доходностью (следующее значение делим на предыдущее)
    n = len(a.CLOSE) # количество строк в таблице 
    mean = a['return'][n - 10:n].mean() # скользящее среднее в колонке return 

    if account_info['assets'] == 0:
        if mean == 0: # если среднее равно нулю, то ничего не делаем в силу неопределённости
            action = pd.DataFrame({'action': [0],
                                   'orderno': [0],
                                   'buysell': [0],
                                   'price': [0],
                                   'volume': [0]})
            orders = {'ACTION': action, 'temp_data': 0}
        else:
            if mean > 0: # если среднее больше нуля, то покупаем в количестве, равном 100
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['B'],
                                       'price': [0],
                                       'volume': [100]})
                TP = a.loc[n - 1, 'CLOSE'] * 1.01 # если цена выросла более чем на 1%, то фиксируем прибыль
                SL = a.loc[n - 1, 'CLOSE'] * 0.99 # если цена упала более чем на 1%, то фиксируем убыток
                temp_data = {'TP': TP, 'SL': SL}
                orders = {'ACTION': action, 'temp_data': temp_data}
            if mean < 0:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['S'],
                                       'price': [0],
                                       'volume': [100]})
                TP = a.loc[n - 1, 'CLOSE'] * 1.01 # если цена выросла более чем на 1%, то фиксируем прибыль
                SL = a.loc[n - 1, 'CLOSE'] * 0.99 # если цена упала более чем на 1%, то фиксируем убыток
                temp_data = {'TP': TP, 'SL': SL} # таблица с take profit и stop loss
                orders = {'ACTION': action, 'temp_data': temp_data}

    else:
        if account_info['assets'] > 0:
            if a.loc[n - 1, 'CLOSE'] > temp_data['TP'] or a.loc[n - 1, 'CLOSE'] < temp_data['SL']:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['S'],
                                       'price': [0],
                                       'volume': [100]})
                orders = {'ACTION': action, 'temp_data': temp_data}
            else:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': [0],
                                       'price': [0],
                                       'volume': [0]})
                orders = {'ACTION': action, 'temp_data': temp_data}
        if account_info['assets'] < 0:
            if a.loc[n - 1, 'CLOSE'] > temp_data['TP'] or a.loc[n - 1, 'CLOSE'] < temp_data['SL']:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['B'],
                                       'price': [0],
                                       'volume': [100]})
                orders = {'ACTION': action, 'temp_data': temp_data}
            else:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': [0],
                                       'price': [0],
                                       'volume': [0]})
                orders = {'ACTION': action, 'temp_data': temp_data}
    return orders

**Бот моментум-реверс по рыночным заявкам**

In [28]:
import pandas as pd
# import numpy as np
# import math
# import matplotlib.pyplot as plt
# import sklearn.linear_model as skl
# import datetime as dt
# import os


def strategy_R(hist_price, bid_ask, account_info=0,temp_data=0):
    a = hist_price # для компактности заменям hist_price на a
    a['return'] = a['CLOSE'] / a['CLOSE'].shift() # добавили колонку с доходностью (следующее значение делим на предыдущее)
    n = len(a.CLOSE) # количество строк в таблице 
    mean = a['return'][n - 20:n].mean() # скользящее среднее в колонке return 

    if account_info['assets'] == 0:
        if mean == 0: # если среднее равно нулю, то ничего не делаем в силу неопределённости
            action = pd.DataFrame({'action': [0],
                                   'orderno': [0],
                                   'buysell': [0],
                                   'price': [0],
                                   'volume': [0]})
            orders = {'ACTION': action, 'temp_data': 0}
        else:
            if mean > 0: # если среднее больше нуля, то продаем в количестве, равном 100
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['S'],
                                       'price': [0],
                                       'volume': [100]})
                TP = a.loc[n - 1, 'CLOSE'] * 1.01 # если цена выросла более чем на 1%, то фиксируем прибыль
                SL = a.loc[n - 1, 'CLOSE'] * 0.99 # если цена упала более чем на 1%, то фиксируем убыток
                temp_data = {'TP': TP, 'SL': SL}
                orders = {'ACTION': action, 'temp_data': temp_data}
            if mean < 0:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['B'],
                                       'price': [0],
                                       'volume': [100]})
                TP = a.loc[n - 1, 'CLOSE'] * 1.01 # если цена выросла более чем на 1%, то фиксируем прибыль
                SL = a.loc[n - 1, 'CLOSE'] * 0.99 # если цена упала более чем на 1%, то фиксируем убыток
                temp_data = {'TP': TP, 'SL': SL} # таблица с take profit и stop loss
                orders = {'ACTION': action, 'temp_data': temp_data}

    else:
        if account_info['assets'] > 0:
            if a.loc[n - 1, 'CLOSE'] > temp_data['TP'] or a.loc[n - 1, 'CLOSE'] < temp_data['SL']:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['B'],
                                       'price': [0],
                                       'volume': [100]})
                orders = {'ACTION': action, 'temp_data': temp_data}
            else:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': [0],
                                       'price': [0],
                                       'volume': [0]})
                orders = {'ACTION': action, 'temp_data': temp_data}
        if account_info['assets'] < 0:
            if a.loc[n - 1, 'CLOSE'] > temp_data['TP'] or a.loc[n - 1, 'CLOSE'] < temp_data['SL']:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['S'],
                                       'price': [0],
                                       'volume': [100]})
                orders = {'ACTION': action, 'temp_data': temp_data}
            else:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': [0],
                                       'price': [0],
                                       'volume': [0]})
                orders = {'ACTION': action, 'temp_data': temp_data}
    return orders

**Робот 4**

In [29]:
def strategy_R2(hist_price, bid_ask, account_info=0,temp_data=0):
    a = hist_price # для компактности заменям hist_price на a
    a['return'] = a['CLOSE'] / a['CLOSE'].shift() # добавили колонку с доходностью (следующее значение делим на предыдущее)
    n = len(a.CLOSE) # количество строк в таблице 
    mean = a['return'][n - 15:n].mean() # скользящее среднее в колонке return 

    if account_info['assets'] == 0:
        if mean == 0: # если среднее равно нулю, то ничего не делаем в силу неопределённости
            action = pd.DataFrame({'action': [0],
                                   'orderno': [0],
                                   'buysell': [0],
                                   'price': [0],
                                   'volume': [0]})
            orders = {'ACTION': action, 'temp_data': 0}
        else:
            if mean > 0: # если среднее больше нуля, то продаем в количестве, равном 100
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['S'],
                                       'price': [0],
                                       'volume': [100]})
                TP = a.loc[n - 1, 'CLOSE'] * 1.01 # если цена выросла более чем на 1%, то фиксируем прибыль
                SL = a.loc[n - 1, 'CLOSE'] * 0.99 # если цена упала более чем на 1%, то фиксируем убыток
                temp_data = {'TP': TP, 'SL': SL}
                orders = {'ACTION': action, 'temp_data': temp_data}
            if mean < 0:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['B'],
                                       'price': [0],
                                       'volume': [100]})
                TP = a.loc[n - 1, 'CLOSE'] * 1.01 # если цена выросла более чем на 1%, то фиксируем прибыль
                SL = a.loc[n - 1, 'CLOSE'] * 0.99 # если цена упала более чем на 1%, то фиксируем убыток
                temp_data = {'TP': TP, 'SL': SL} # таблица с take profit и stop loss
                orders = {'ACTION': action, 'temp_data': temp_data}

    else:
        if account_info['assets'] > 0:
            if a.loc[n - 1, 'CLOSE'] > temp_data['TP'] or a.loc[n - 1, 'CLOSE'] < temp_data['SL']:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['B'],
                                       'price': [0],
                                       'volume': [100]})
                orders = {'ACTION': action, 'temp_data': temp_data}
            else:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': [0],
                                       'price': [0],
                                       'volume': [0]})
                orders = {'ACTION': action, 'temp_data': temp_data}
        if account_info['assets'] < 0:
            if a.loc[n - 1, 'CLOSE'] > temp_data['TP'] or a.loc[n - 1, 'CLOSE'] < temp_data['SL']:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': ['S'],
                                       'price': [0],
                                       'volume': [100]})
                orders = {'ACTION': action, 'temp_data': temp_data}
            else:
                action = pd.DataFrame({'action': [0],
                                       'orderno': [0],
                                       'buysell': [0],
                                       'price': [0],
                                       'volume': [0]})
                orders = {'ACTION': action, 'temp_data': temp_data}
    return orders

# **Непосредственно торговая площадка**

**Функция queueing**

In [30]:
def group_mutate_ungroup(df, column_of_values, id_name):
    """
    Это промежуточная функция, нужна для того, чтобы заменить несколько строк в R
    В дальнейшем используется внутри queueing
    """
    df_ = df.sort_values(by = [column_of_values])
    nums = []
    for val in np.unique(df_[column_of_values]):
        nums += range(1, len(df_[df_[column_of_values] == val]) + 1)
    df_[id_name] = nums
    return df_

def queueing(hist_price, bid_ask, accounts_info, temp_data):
    
    # Создаем список с функциями юзеров
    robots = [strategy_M,strategy_M2,strategy_R,strategy_R2]
    robot = {}
    i=0
    for user in USERS:
        robot.update({user:robots[i]})
        i=i+1
    
    List_of_Actions = { user: 0 for user in USERS}
    
       
    # Создаем пустой датафрейм-заготовку
    df_w = pd.DataFrame({'action': [], 'orderno': [], 'buysell': [], 'price': [], 'volume': [], 'ID': [] })
    
    for user in USERS:
        # выполняем код одного из роботов (вообще, надо бы переписать, походу, т.к. имена юзаров будут меняться)
        result = robot[user](hist_price, bid_ask,
                            Accounts_Info[user], temp_data[user])
        print(result)
        # вытаскиваем таблицу экшен для данного юзера
        List_of_Actions[user] = result['ACTION']
        # проставляем ему айди во все строки
        List_of_Actions[user]['ID'] = user
        # и уже вместе с айди добавляем в общую таблицу всех экшен
        df_w = df_w.append(List_of_Actions[user])
        # вытаскиваем темп_дату
        temp_data[user] = result['temp_data']
        
    # создаем список переменных, которые потом выйдут как результат функции
    OrderQueue = {'market': 0, 'change': 0, 'limit': 0}
    
    # рыночные заявки (ненулевые)
    market = df_w[df_w['action'] == 0].dropna(subset=['buysell', 'volume'])
    market = market[market['volume'] > 0]
    
    if len(market != 0):
        # упорядочиваем их в разрезе каждого юзера
        market = group_mutate_ungroup(market, "ID", 'no')
        market = market.sort_values(by = ['no', 'ID'])
        market = market[['no', 'ID', 'buysell', 'volume']]
        
    # добавляем в результат функции
    OrderQueue['market'] = market
    
    """
    Далее то же самое для лимитных заявок и изменений
    Порядок действий аналогичен рыночным заявкам с небольшими разницами
    """
    
    change = df_w[df_w['action'] == 2].dropna(subset=['orderno', 'buysell', 'price', 'volume'])
    change = change[(change['volume'] > 0) & (change['price'] > 0)]
    change = change.round({'price': 2})
    
    if len(change) != 0:
        change = group_mutate_ungroup(change, "ID", 'no')
        change = change.sort_values(by = ['no', 'ID'])
        change = change[['no', 'ID', 'orderno', 'buysell', 'price', 'volume']]
    OrderQueue['change'] = change
    
    limit = df_w[df_w['action'] == 1].dropna(subset=['buysell', 'price', 'volume'])
    limit = limit[(limit['volume'] > 0) & (limit['price'] > 0)]
    limit = limit.round({'price': 2})
    
    if len(limit) != 0:
        limit = group_mutate_ungroup(limit, "ID", 'no')
        limit = limit.sort_values(by = ['no', 'ID'])
        limit = limit[['no', 'ID', 'buysell', 'price', 'volume']]
    OrderQueue['limit'] = limit
    
#     temp_data = temp_data # интерпретация команды temp_data <<- temp_data (для чего вообще эта команда?)
    
    print(OrderQueue)
    return OrderQueue, temp_data

**Функция user_trading**

In [31]:
def user_trading(queue, TIME, TRADE_SEC, last_book, last_trade, OrderBook, TradeBook, DOM):
 
    ### MARKET ###
    
    df_w = queue['market']
    if len(df_w) > 0:
        for i in range(1,len(df_w)):
            user_order = df_w.iloc[i]
            # следующая строка нужна, если бы мы использовали iloc, но т.к. мы просто делаем append, она по сути не нужна
            # i_book = len(OrderBook[OrderBook['no'] != 0]) + 1
            
            
            # добавляем в ордербук новую строку по этой заявке
            OrderBook = OrderBook.append(pd.DataFrame({'no': [np.append(OrderBook['no'].values, np.array(last_book)).max() + 1],
                                'time': [TIME],
                                'trade_sec': [TRADE_SEC],
                                'ID': [user_order['ID']],
                                'action': ['set'],
                                'buysell': [user_order['buysell']],
                                'orderno': [0],
                                'bot_orderno': [0],
                                'price': [0],
                                'volume': [user_order['volume']]
                               })).reset_index(drop=True)
            
            if user_order['buysell'] == "B":
                while user_order['volume'] > 0 and len(DOM['supply']) != 0:
                    # добавляем новую строку в трейдбук
                    vol = np.array([DOM['supply'].loc[0, 'volume'], user_order['volume']]).min()
                    TradeBook = TradeBook.append(pd.DataFrame({'tradeno': [np.append(TradeBook['tradeno'].values, np.array(last_trade)).max() + 1],
                                'time': [TIME],
                                'trade_sec': [TRADE_SEC],
                                'buy_ID': [user_order['ID']],
                                'sell_ID': [DOM['supply'].loc[0, 'ID']],
                                'buy_orderno': [0],
                                'sell_orderno': [DOM['supply'].loc[0, 'orderno']],
                                'price': [DOM['supply'].loc[0, 'price']],
                                'volume': [vol]
                               })).reset_index(drop=True)
                    
                    OrderBook = OrderBook.append(pd.DataFrame({'no': [np.append(OrderBook['no'].values, np.array(last_book)).max() + 1,
                                                         np.append(OrderBook['no'].values, np.array(last_book)).max() + 2],
                                'time': 2 * [TIME],
                                'trade_sec': 2 * [TRADE_SEC],
                                'ID': [user_order['ID'], DOM['supply'].loc[0, 'ID']],
                                'action': 2 * ['trade'],
                                'buysell': ['B', 'S'],
                                'orderno': [0, 0],
                                'bot_orderno': [0, 0],
                                'price': 2 * [DOM['supply'].loc[0, 'price']],
                                'volume': 2 * [vol]
                               })).reset_index(drop=True)
                    # снижаем объемы заявок на обхем сделки
                    df_w.iloc[1, df_w.columns.get_loc('volume')] -= vol
                    
                    DOM['supply'].loc[0, 'volume'] -= vol
                    
                    DOM['supply'] = DOM['supply'][DOM['supply']['volume'] != 0].sort_values(by = ['price', 'orderno']).reset_index(drop=True)

                if len(DOM['supply']) == 0:
                    DOM['supply'] = pd.DataFrame({'orderno': [0], 'bot_orderno': [0], 'ID': ["bot"],
                               'price': [0], 'volume': [0]}).reset_index(drop=True)
                                   
                    
            if user_order['buysell'] == "S":
                while user_order['volume'] > 0 and len(DOM['demand']) != 0:
                    # добавляем новую строку в трейдбук
                    vol = np.array([DOM['demand'].loc[0, 'volume'], user_order['volume']]).min()
                    TradeBook = TradeBook.append(pd.DataFrame({'tradeno': [np.append(TradeBook['tradeno'].values, np.array(last_book)).max() + 1],
                                'time': [TIME],
                                'trade_sec': [TRADE_SEC],
                                'buy_ID': [DOM['demand'].loc[0, 'ID']],
                                'sell_ID': [user_order['ID']],
                                'buy_orderno': [DOM['demand'].loc[0, 'orderno']],
                                'sell_orderno': [0],
                                'price': [DOM['demand'].loc[0, 'price']],
                                'volume': [vol]
                               })).reset_index(drop=True)
                    
                    OrderBook = OrderBook.append(pd.DataFrame({'no': [np.append(OrderBook['no'].values, np.array(last_book)).max() + 1,
                                                         np.append(OrderBook['no'].values, np.array(last_book)).max() + 2],
                                'time': 2 * [TIME],
                                'trade_sec': 2 * [TRADE_SEC],
                                'ID': [DOM['demand'].loc[0, 'ID'], user_order['ID']],
                                'action': 2 * ['trade'],
                                'buysell': ['B', 'S'],
                                'orderno': [0, 0],
                                'bot_orderno': [0, 0],
                                'price': 2 * [DOM['demand'].loc[0, 'price']],
                                'volume': 2 * [vol]
                               })).reset_index(drop=True)
                    
                    # снижаем объемы заявок на обхем сделки
                    df_w.iloc[1, df_w.columns.get_loc('volume')] -= vol
                    DOM['demand'].loc[0, 'volume'] -= vol
                    
                    DOM['demand'] = DOM['demand'][DOM['demand']['volume'] != 0].sort_values(by = ['price', 'orderno'], ascending = [False, True]).reset_index(drop=True)
                    
                if len(DOM['demand']) == 0:
                    DOM['demand'] = pd.DataFrame({'orderno': [0], 'bot_orderno': [0], 'ID': ["bot"],
                               'price': [0], 'volume': [0]}).reset_index(drop=True)
    ### LIMIT CHANGE ###
    
    df_w = queue['change']
    if len(df_w) > 0:
        for i in range(1,len(df_w)):
            user_order = df_w.iloc[i]
            
            d = OrderBook.iloc[len(OrderBook) - 1, OrderBook.columns.get_loc('buysell')]
            a = DOM['demand'][DOM['demand']['bot_orderno'] == bot_order['ORDERNO']] if d == 'B' else DOM['supply'][DOM['supply']['bot_orderno'] == bot_order['ORDERNO']]
            
            if len(a) != 0: 
            
                
                orderno = 0

                # добавляем в ордербук новую строку по этой заявке
                OrderBook = OrderBook.append(pd.DataFrame({'no': [np.append(OrderBook['no'].values, np.array(last_book)).max() + 1],
                                    'time': [TIME],
                                    'trade_sec': [TRADE_SEC],
                                    'ID': [user_order['ID']],
                                    'action': ['set'],
                                    'buysell': [user_order['buysell']],
                                    'orderno': [orderno],
                                    'bot_orderno': [0],
                                    'price': [user_order['price']],
                                    'volume': [user_order['volume']]
                                   })).reset_index(drop=True)

                if bot_order['BUYSELL'] == 'B':
                    a = DOM['demand']
                    a['price'] = (a['orderno'] != orderno) * a['price'] + (a['orderno'] == orderno) * user_order['price']
                    a['volume'] = (a['orderno'] != orderno) * a['volume'] + (a['orderno'] == orderno) * user_order['volume']
                    DOM['demand'] = a['volume' != 0].sort_values(by = ['price', 'orderno'], ascending = [False, True]).reset_index(drop=True)
                if bot_order['BUYSELL'] == 'S':
                    a = DOM['supply']
                    a['price'] = (a['orderno'] != orderno) * a['price'] + (a['orderno'] == orderno) * user_order['price']
                    a['volume'] = (a['orderno'] != orderno) * a['volume'] + (a['orderno'] == orderno) * user_order['volume']
                    DOM['supply'] = a['volume' != 0].sort_values(by = ['price', 'orderno'], ascending = [False, True]).reset_index(drop=True)

                if len(DOM['demand']) == 0:
                    DOM['demand'] = pd.DataFrame({'orderno': [0], 'bot_orderno': [0], 'ID': ["bot"],
                                   'price': [0], 'volume': [0]}).reset_index(drop=True)
                if len(DOM['supply']) == 0:
                    DOM['supply'] = pd.DataFrame({'orderno': [0], 'bot_orderno': [0], 'ID': ["bot"],
                                       'price': [0], 'volume': [0]}).reset_index(drop=True)
    
    ### LIMIT SET ###
    
    df_w = queue['limit']
    if len(df_w) > 0:
        for i in range(1,len(df_w)):
            user_order = df_w.iloc[i]
            print('qwerty')
            orderno = OrderBook['orderno'].values.max() + 1
            
            # добавляем в ордербук новую строку по этой заявке
            OrderBook = OrderBook.append(pd.DataFrame({'no': [np.append(OrderBook['no'].values, np.array(last_book)).max() + 1],
                                'time': [TIME],
                                'trade_sec': [TRADE_SEC],
                                'ID': [user_order['ID']],
                                'action': ['set'],
                                'buysell': [user_order['buysell']],
                                'orderno': [orderno],
                                'bot_orderno': [0],
                                'price': [user_order['price']],
                                'volume': [user_order['volume']]
                               })).reset_index(drop=True)
            
            if user_order['buysell'] == 'B':
                DOM['demand'] = DOM['demand'].append(pd.DataFrame({'orderno': [orderno],
                                'bot_orderno': [0],
                                'ID': [user_order['ID']],
                                'price': [user_order['price']],
                                'volume': [user_order['volume']]
                               })).reset_index(drop=True)
                DOM['demand'] = DOM['demand'][DOM['demand']['volume'] != 0].sort_values(by = ['price', 'orderno'],
                                                                                       ascending = [False, True]).reset_index(drop=True)              
            if user_order['buysell'] == 'S':
                DOM['supply'] = DOM['supply'].append(pd.DataFrame({'orderno': [orderno],
                                'bot_orderno': [0],
                                'ID': [user_order['ID']],
                                'price': [user_order['price']],
                                'volume': [user_order['volume']]
                               })).reset_index(drop=True)
                DOM['supply'] = DOM['supply'][DOM['supply']['volume'] != 0].sort_values(by = ['price', 'orderno']).reset_index(drop=True)
            
            if len(DOM['supply']) == 0:
                DOM['supply'] = pd.DataFrame({'orderno': [0], 'bot_orderno': [0], 'ID': ["bot"],
                                       'price': [0], 'volume': [0]}).reset_index(drop=True)
                
            if len(DOM['demand']) == 0:
                DOM['demand'] = pd.DataFrame({'orderno': [0], 'bot_orderno': [0], 'ID': ["bot"],
                                       'price': [0], 'volume': [0]}).reset_index(drop=True)
            
            while DOM['demand'].loc[0, 'price'] >= DOM['supply'].loc[0, 'price'] and len(DOM['demand'] != 0) and len(DOM['supply'] != 0):
                
                print('sdjhaldkfjlasdf')
                
                # пока выполняется пересечение (смыкание) бид-аск спреда -- торгуем
                # добавляем новую строку в трейдбук
                vol = np.array([DOM['demand'].loc[0, 'volume'], DOM['supply'].loc[0, 'volume']]).min()
                TradeBook = TradeBook.append(pd.DataFrame({'tradeno': [np.append(TradeBook['tradeno'].values, np.array(last_trade)).max() + 1],
                            'time': [TIME],
                            'trade_sec': [TRADE_SEC],
                            'buy_ID': [DOM['demand'].loc[0, 'ID']],
                            'sell_ID': [DOM['supply'].loc[0, 'ID']],
                            'buy_orderno': [DOM['demand'].loc[0, 'orderno']],
                            'sell_orderno': [DOM['supply'].loc[0, 'orderno']],
                            # цена сделки это цена заявки, которая была раньше
                            'price': [DOM['demand'].loc[0, 'price'] if DOM['demand'].loc[0, 'orderno'] < DOM['supply'].loc[0, 'orderno'] else DOM['supply'].loc[0, 'price']],
                            # объем сделки это объем наименьшей заявки
                            'volume': [vol]
                           })).reset_index(drop=True)
                OrderBook = OrderBook.append(pd.DataFrame({'no': [np.append(OrderBook['no'].values, np.array(last_book)).max() + 1,
                                                     np.append(OrderBook['no'].values, np.array(last_book)).max() + 2],
                                'time': 2 * [TIME],
                                'trade_sec': 2 * [TRADE_SEC],
                                'ID': [DOM['demand'].loc[0, 'ID'], DOM['supply'].loc[0, 'ID']],
                                'action': 2 * ['trade'],
                                'buysell': ['B', 'S'],
                                'orderno': [DOM['demand'].loc[0, 'orderno'], DOM['supply'].loc[0, 'orderno']],
                                'bot_orderno': [0, 0],
                                'price': 2 * [DOM['demand'].loc[0, 'price'] if DOM['demand'].loc[0, 'orderno'] < DOM['supply'].loc[0, 'orderno'] else DOM['supply'].loc[0, 'price']],
                                'volume': 2 * [vol]
                               })).reset_index(drop=True)
                
                DOM['demand'].loc[0, 'volume'] -= vol
                DOM['supply'].loc[0, 'volume'] -= vol
                
                DOM['demand'] = DOM['demand'][DOM['demand']['volume'] != 0].sort_values(by = ['price', 'orderno'],
                                                                                       ascending = [False, True]).reset_index(drop=True)
                DOM['supply'] = DOM['supply'][DOM['supply']['volume'] != 0].sort_values(by = ['price', 'orderno']).reset_index(drop=True)
                

            if len(DOM['supply']) == 0:
                DOM['supply'] = pd.DataFrame({'orderno': [0], 'bot_orderno': [0], 'ID': ["bot"],
                                   'price': [0], 'volume': [0]}).reset_index(drop=True)

            if len(DOM['demand']) == 0:
                DOM['demand'] = pd.DataFrame({'orderno': [0], 'bot_orderno': [0], 'ID': ["bot"],
                                   'price': [0], 'volume': [0]}).reset_index(drop=True)
                
    # return {'OrderBook': OrderBook, 'TradeBook': TradeBook, 'DOM': DOM}
    return OrderBook, TradeBook, DOM

**Функция bot_trading**

In [51]:
def bot_trading(trade_sec, OrderBook, TradeBook, DOM, last_book = 482114, last_trade = 122719):

    df_w = Big_OrderBook[Big_OrderBook['TIME_FROM_START'] == trade_sec]
    if len(df_w) == 0: return OrderBook, TradeBook, DOM
 
    for i in range(0,len(df_w)):
        bot_order = df_w.iloc[i]

        orderno = OrderBook['orderno'].values.max() + 1    
        if bot_order['ACTION'] == 1:             
            OrderBook = OrderBook.append(pd.DataFrame({'no': [np.append(OrderBook['no'].values, np.array(last_no_book)).max() + 1],
                            'time': [bot_order['TIME_FROM_START']],
                            'trade_sec': [bot_order['SECCODE']],
                            'ID': ['bot'],
                            'action': ['set'],
                            'buysell': [bot_order['BUYSELL']],
                            'orderno': [orderno],
                            'bot_orderno': [bot_order['ORDERNO']],
                            'price': [bot_order['PRICE']],
                            'volume': [bot_order['VOLUME']]
                           })).reset_index(drop = True)
            OrderBook

            if bot_order['BUYSELL'] == 'B':
                DOM['demand'] = DOM['demand'].append(pd.DataFrame({'orderno': [orderno],
                                'bot_orderno': [bot_order['ORDERNO']],
                                'ID': ['bot'],
                                'price': [bot_order['PRICE']],
                                'volume': [bot_order['VOLUME']]
                               })).reset_index(drop=True)
                DOM['demand'] = DOM['demand'][DOM['demand']['volume'] != 0].sort_values(by = ['price', 'orderno'],
                                                                                       ascending = [False, True]).reset_index(drop=True)
            if bot_order['BUYSELL'] == 'S':
                DOM['supply'] = DOM['supply'].append(pd.DataFrame({'orderno': [orderno],
                                'bot_orderno': [bot_order['ORDERNO']],
                                'ID': ['bot'],
                                'price': [bot_order['PRICE']],
                                'volume': [bot_order['VOLUME']]
                               })).reset_index(drop=True)
                DOM['supply'] = DOM['supply'][DOM['supply']['volume'] != 0].sort_values(by = ['price', 'orderno']).reset_index(drop=True)
                
            if len(DOM['supply']) == 0:
                    DOM['supply'] = pd.DataFrame({'orderno': [0], 'bot_orderno': [0], 'ID': ["bot"], 'price': [0], 'volume': [0]}).reset_index(drop=True)
            
            if len(DOM['demand']) == 0:
                    DOM['demand'] = pd.DataFrame({'orderno': [0], 'bot_orderno': [0], 'ID': ["bot"], 'price': [0], 'volume': [0]}).reset_index(drop=True)
            
            while len(DOM['supply'] != 0) and len(DOM['demand'] != 0) and DOM['demand'].loc[0, 'price'] >= DOM['supply'].loc[0, 'price'] :
                # добавляем новую строку в трейдбук
                vol = np.array([DOM['demand'].loc[0, 'volume'], DOM['supply'].loc[0, 'volume']]).min()
                TradeBook = TradeBook.append(pd.DataFrame({'tradeno': [np.append(TradeBook['tradeno'].values, np.array(last_trade)).max() + 1],
                            'time': [bot_order['TIME_FROM_START']],
                            'trade_sec': [bot_order['SECCODE']],
                            'buy_ID': [DOM['demand'].loc[0, 'ID']],
                            'sell_ID': [DOM['supply'].loc[0, 'ID']],
                            'buy_orderno': [DOM['demand'].loc[0, 'orderno']],
                            'sell_orderno': [DOM['supply'].loc[0, 'orderno']],
                            'price': [DOM['demand'].loc[0, 'price'] if DOM['demand'].loc[0, 'orderno'] < DOM['supply'].loc[0, 'orderno'] else DOM['supply'].loc[0, 'price']],
                            'volume': [vol]
                            })).reset_index(drop=True)
                
                OrderBook = OrderBook.append(pd.DataFrame({'no': [np.append(OrderBook['no'].values, np.array(last_book)).max() + 1,
                                     np.append(OrderBook['no'].values, np.array(last_book)).max() + 2],
                            'time': 2 * [bot_order['TIME_FROM_START']],
                            'trade_sec': 2 * [bot_order['SECCODE']],
                            'ID': [DOM['demand'].loc[0, 'ID'], DOM['supply'].loc[0, 'ID']],
                            'action': 2 * ['trade'],
                            'buysell': ['B', 'S'],
                            'orderno': [DOM['demand'].loc[0, 'orderno'], DOM['supply'].loc[0, 'orderno']],
                            'bot_orderno': [0, 0],
                            'price': 2 * [DOM['demand'].loc[0, 'price'] if DOM['demand'].loc[0, 'orderno'] < DOM['supply'].loc[0, 'orderno'] else DOM['supply'].loc[0, 'price']],
                            'volume': 2 * [vol]
                            })).reset_index(drop=True)
                DOM['demand'].loc[0, 'volume'] -= vol
                DOM['supply'].loc[0, 'volume'] -= vol
                
                DOM['demand'] = DOM['demand'][DOM['demand']['volume'] != 0].sort_values(by = ['price', 'orderno'],
                                                                                       ascending = [False, True]).reset_index(drop=True)
                DOM['supply'] = DOM['supply'][DOM['supply']['volume'] != 0].sort_values(by = ['price', 'orderno']).reset_index(drop=True)
                
                
                
            if len(DOM['demand']) == 0:
                DOM['demand'] = pd.DataFrame({'orderno': [0], 'bot_orderno': [0], 'ID': ["bot"], 'price': [0], 'volume': [0]}).reset_index(drop=True)

            if len(DOM['supply']) == 0:
                DOM['supply'] = pd.DataFrame({'orderno': [0], 'bot_orderno': [0], 'ID': ["bot"], 'price': [0], 'volume': [0]}).reset_index(drop=True)
            
        if bot_order['ACTION'] == 0:
            # проверяем на наличие заявки в стакане
            d = OrderBook.iloc[len(OrderBook) - 1, OrderBook.columns.get_loc('buysell')]
            a = DOM['demand'][DOM['demand']['bot_orderno'] == bot_order['ORDERNO']] if d == 'B' else DOM['supply'][DOM['supply']['bot_orderno'] == bot_order['ORDERNO']]
            
            if len(a) != 0:                                                                                          
               
                OrderBook = OrderBook.append(pd.DataFrame({'no': [np.append(OrderBook['no'].values, np.array(last_book)).max() + 1],
                                    'time': [bot_order['TIME_FROM_START']],
                                    'trade_sec': [bot_order['SECCODE']],
                                    'ID': ['bot'],
                                    'action': ['drop'],
                                    'buysell': [bot_order['BUYSELL']],
                                    'orderno': [orderno],
                                    'bot_orderno': [bot_order['ORDERNO']],
                                    'price': [bot_order['PRICE']],
                                    'volume': [bot_order['VOLUME']]
                                   }))                          
                if bot_order['BUYSELL'] == 'B':
                    DOM['demand'] = DOM['demand'][DOM['demand']['orderno'] != orderno].reset_index(drop=True)

                if bot_order['BUYSELL'] == 'S':
                    DOM['supply'] = DOM['supply'][DOM['supply']['orderno'] != orderno].reset_index(drop=True)
                
    return OrderBook, TradeBook, DOM

# **Площадка**

In [53]:
import time
import numpy as np

#Запускается торговый цикл
# OrderBook, TradeBook, DOM = rm_dom(3e4, 7e3) 

last_no_book = 0
last_no_trade = 0
T_SEC_LIMIT = 1000005 * 2
a = np.unique(Big_OrderBook[(Big_OrderBook['ACTION'] == 1) | (Big_OrderBook['ACTION'] == 0)]['TIME_FROM_START'])

# for t_sec in range(1000005 - 1, T_SEC_LIMIT):
for t_sec in a:
    t_min = 2080 + t_sec // 60
    print(bid_ask.iloc[0, 2], bid_ask.iloc[0, 1])
    print(t_sec)
    # unknown function
    OrderBook, TradeBook, DOM = bot_trading(t_sec,  OrderBook, TradeBook, DOM, last_no_book, last_no_trade)
#     print(TradeBook)
    last_no_book = max(OrderBook['no'])
    last_no_trade = max(TradeBook['tradeno'])

    bid_ask = pd.DataFrame({'time': [t_sec],
                            'bid': [DOM['demand']['price'].max()],
                            'ask': [DOM['supply']['price'].min()]})
    if t_sec // 60 == 0:
        print(t_min)
        print(bid_ask)


    
    queue, temp_data = queueing(hist_price, bid_ask, Accounts_Info, temp_data)
    OrderBook, TradeBook, DOM = user_trading(queue, t_min, t_sec, last_no_book, last_no_trade, OrderBook, TradeBook, DOM)
#     print(TradeBook)
#     print(Accounts_Info)
    # 
    last_no_book = max(OrderBook['no'])
    last_no_trade = max(TradeBook['tradeno'])

    # Определяются заявки на покупку
    x = TradeBook[(TradeBook['buy_ID'] != "X") & (TradeBook['buy_ID'] != "bot")
                  & (TradeBook['trade_sec'] == t_sec)]
    x['cost'] = x['price'] * x['volume']
    x = x.groupby('buy_ID', axis = 0).sum()
    x['cost'] = x['cost'] * -1
    
    x = x.rename(columns = {'cost': 'buy_cost',
                            'volume': 'buy_vol',
                            'buy_ID': 'ID'})
    x.index.names = ['ID']

    # Определяются заявки на продажу
    y = TradeBook[(TradeBook['sell_ID'] != "X") & (TradeBook['sell_ID'] != "bot")
                  & (TradeBook['trade_sec'] == t_sec)]
    y['cost'] = y['price'] * y['volume']
    y = y.groupby('sell_ID', axis = 0).sum()
    y['volume'] = y['volume'] * -1
   
    y = y.rename(columns = {'cost': 'sell_cost',
                            'volume': 'sell_vol',
                            'sell_ID': 'ID'})
    
    y.index.names = ['ID']
    # Объединяются заявок на продажу и покупку
    df_acc_mov = pd.merge(x, y, how = 'outer', on = 'ID')

    # Добавляются USERS
    df_acc_mov = pd.merge(df_acc_mov, pd.DataFrame({'ID': USERS}), how = 'outer', on = 'ID')
    df_acc_mov = df_acc_mov.fillna(0)
    # Рассчитыаются изменения произошедшие по счетам (деньги и активы)
    df_acc_mov['cash_change'] = df_acc_mov['buy_cost'] + df_acc_mov['sell_cost']
    df_acc_mov['assets_change'] = df_acc_mov['buy_vol'] + df_acc_mov['sell_vol']
    df_acc_mov.index = df_acc_mov['ID']
    df_acc_mov = df_acc_mov[['ID', 'cash_change', 'assets_change']]

    # Обновляются данные для каждого робота
    for user in USERS:
        # данные о денежных средствах и активах
        Accounts_Info[user]['cash'] += df_acc_mov.loc[user, 'cash_change']
        Accounts_Info[user]['assets'] += df_acc_mov.loc[user, 'assets_change']

        # Обновление данных по лимитным ордерам
        # Берутся данные отдельно по спросу и отдельно по предложению для пользователей
        dem = DOM['demand']
        dem = dem[dem['ID'] == user]
        dem['buysell'] = 'B'
        sup = DOM['supply']
        sup = sup[sup['ID'] == user]
        sup['buysell'] = 'S'
        
        # Объединяем и переназначем индекс
        dem_sup = pd.concat([dem, sup], keys=['dem', 'sup'])
        dem_sup.index = range(0, len(dem_sup))

        # Добавляется эта информация в один датафрейм
        Accounts_Info[user]['limit_orders'] = dem_sup

        # При отсутствии лимитных заявок, датафрейм заполняется пустыми массивами
        if len(Accounts_Info[user]['limit_orders']) == 0:
            Accounts_Info[user]['limit_orders'] = pd.DataFrame({
                'orderno': [], 'buysell': [],'price': [], 'volume': []
            })
    
    if t_sec % 60 == 0:
        # Задаётся новая строка для последней минуты
        try:
            hist_price.loc[t_min + 1, 'TIME'] = t_min + 1
            hist_price.loc[t_min + 1, 'OPEN'] = TradeBook[TradeBook['time'] == t_min]['price'].values[1]
            hist_price.loc[t_min + 1, 'LOW'] = TradeBook[TradeBook['time'] == t_min]['price'].values.min()
            hist_price.loc[t_min + 1, 'HIGH'] = TradeBook[TradeBook['time'] == t_min]['price'].values.max()
            hist_price.loc[t_min + 1, 'CLOSE'] = TradeBook[TradeBook['time'] == t_min]['price'].values[-1]
        except:
            hist_price.loc[t_min + 1, 'TIME'] = t_min + 1
            hist_price.loc[t_min + 1, 'OPEN'] = 0
            hist_price.loc[t_min + 1, 'LOW'] = 0
            hist_price.loc[t_min + 1, 'HIGH'] = 0
            hist_price.loc[t_min + 1, 'CLOSE'] = 0

    if t_sec % 60 == 59:
        # Обновляются данные для книги заявок
        print(time.ctime())
        print("OrderBook rows in the minute: {}".format(OrderBook[OrderBook['no'] != 0]['no'].sum()))
        OrderBook[OrderBook['no'] != 0].to_csv(
            "OrderBook_tmin_{}.csv".format(t_min)
        )

        # Обновляется непосредственно книга заявок
        N = 3e4
        empty_array = [0 for i in range(int(N))]
        OrderBook = pd.DataFrame({
            'no': empty_array,
            'time': empty_array,
            'trade_sec': empty_array,
            'ID': empty_array,
            'action': ['NA' for i in range(int(N))],
            'buysell': ['X' for i in range(int(N))],
            'orderno': empty_array,
            'bot_orderno': empty_array,
            'price': empty_array,
            'volume': empty_array
        })

        # Обновляются данные для книги сделок
        print("TradeBook rows in the minute: {}".format(TradeBook[TradeBook['tradeno'] != 0]['tradeno'].sum()))
        TradeBook[TradeBook['tradeno'] != 0].to_csv(
            "TradeBook_tmin_{}.csv".format(t_min)
        )

        # Обновляется непосредственно книга сделок 
        N2 = 7e3
        empty_array2 = [0 for i in range(int(N2))]
        TradeBook = pd.DataFrame({
            'tradeno': empty_array2,
            'time': empty_array2,
            'trade_sec': empty_array2,
            'buy_ID': ['X' for i in range(int(N2))],
            'sell_ID': ['X' for i in range(int(N2))],
            'buy_orderno': empty_array2,
            'sell_orderno': empty_array2,
            'price': empty_array2,
            'volume': empty_array2
        })
    
    # обновление данных для DOM и hist_price
    DOM['supply'].to_csv("DOM_supply_backup.csv")
    DOM['demand'].to_csv("DOM_demand_backup.csv")
    hist_price.to_csv("hist_price_backup.csv")

    if t_sec % 60 == 59:
        print("----------------------------------")



103.45 103.38
0
2080
   time     bid     ask
0     0  103.38  103.45
{'ACTION':    action  orderno buysell  price  volume
0       0        0       B      0     100, 'temp_data': {'TP': 104.5451, 'SL': 102.4749}}
{'ACTION':    action  orderno buysell  price  volume
0       0        0       B      0     100, 'temp_data': {'TP': 104.5451, 'SL': 102.4749}}
{'ACTION':    action  orderno buysell  price  volume
0       0        0       S      0     100, 'temp_data': {'TP': 104.5451, 'SL': 102.4749}}
{'ACTION':    action  orderno buysell  price  volume
0       0        0       S      0     100, 'temp_data': {'TP': 104.5451, 'SL': 102.4749}}
{'market':    no      ID buysell  volume
0   1  User_1       B   100.0
0   1  User_2       B   100.0
0   1  User_3       S   100.0
0   1  User_4       S   100.0, 'change': Empty DataFrame
Columns: [action, orderno, buysell, price, volume, ID]
Index: [], 'limit': Empty DataFrame
Columns: [action, orderno, buysell, price, volume, ID]
Index: []}


KeyboardInterrupt: 